# 半導体サイクルと日本半導体銘柄の株価分析

## 概要
経産省 鉱工業指数（IIP）の電子部品・デバイス工業データと日本半導体関連銘柄の株価を重ね合わせ、
半導体サイクルの局面判定と株価推移の関係を可視化する。

## データソース
- **鉱工業指数**: 経済産業省（出荷指数・在庫指数）
- **株価**: Yahoo Finance（月次終値）
- **SOX指数**: フィラデルフィア半導体指数

## 分析期間
2010年1月 〜 直近

In [ ]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from src.stock_data import (
    SEMICONDUCTOR_STOCKS,
    fetch_stock_prices,
    fetch_sox_index,
    normalize_prices,
)
from src.iip_data import create_sample_iip_data, calc_cycle_indicator

pd.options.display.float_format = "{:.1f}".format
print("Setup complete.")

## 1. 半導体サイクル指標の算出

電子部品・デバイス工業の **出荷指数 / 在庫指数** 比率（SI比率）で
半導体サイクルの局面を4分類する。

| 局面 | SI比率 | 方向 |
|------|--------|------|
| 回復期 | < 1.0 | 上昇 |
| 好況期 | ≥ 1.0 | 上昇 |
| 後退期 | ≥ 1.0 | 下降 |
| 不況期 | < 1.0 | 下降 |

In [ ]:
# IIPデータ取得（サンプルデータ使用）
iip_raw = create_sample_iip_data()
iip = calc_cycle_indicator(iip_raw)

print(f"期間: {iip.index[0].strftime('%Y-%m')} 〜 {iip.index[-1].strftime('%Y-%m')}")
print(f"レコード数: {len(iip)}")
iip.tail()

In [ ]:
# ========================================
# Chart 1: 出荷指数・在庫指数 + SI比率
# ========================================

PHASE_COLORS = {
    "回復期": "rgba(76, 175, 80, 0.15)",   # 緑
    "好況期": "rgba(33, 150, 243, 0.15)",  # 青
    "後退期": "rgba(255, 152, 0, 0.15)",   # 橙
    "不況期": "rgba(244, 67, 54, 0.15)",    # 赤
}

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=(
        "電子部品・デバイス工業 出荷指数・在庫指数",
        "出荷/在庫比率（SI比率）とサイクル局面",
    ),
    row_heights=[0.5, 0.5],
)

# 上段: 出荷・在庫指数
fig.add_trace(
    go.Scatter(
        x=iip.index, y=iip["shipment_index"],
        name="出荷指数", line=dict(color="#2196F3", width=2),
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=iip.index, y=iip["inventory_index"],
        name="在庫指数", line=dict(color="#FF9800", width=2),
    ),
    row=1, col=1,
)

# 下段: SI比率
fig.add_trace(
    go.Scatter(
        x=iip.index, y=iip["si_ratio_ma3"],
        name="SI比率(3MA)", line=dict(color="#673AB7", width=2.5),
    ),
    row=2, col=1,
)

# SI比率=1.0の基準線
fig.add_hline(y=1.0, line_dash="dash", line_color="gray", row=2, col=1)

# サイクル局面の背景色
phases = iip["cycle_phase"]
for phase, color in PHASE_COLORS.items():
    mask = phases == phase
    if not mask.any():
        continue
    starts = mask & ~mask.shift(1, fill_value=False)
    ends = mask & ~mask.shift(-1, fill_value=False)
    for s, e in zip(iip.index[starts], iip.index[ends]):
        fig.add_vrect(
            x0=s, x1=e,
            fillcolor=color, layer="below", line_width=0,
            row=2, col=1,
        )

fig.update_layout(
    height=700, width=1100,
    template="plotly_white",
    title_text="半導体サイクル指標（電子部品・デバイス工業 IIP）",
    title_x=0.5,
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)
fig.update_yaxes(title_text="指数 (2020=100)", row=1, col=1)
fig.update_yaxes(title_text="SI比率", row=2, col=1)

fig.show()

## 2. 半導体銘柄の株価推移

In [ ]:
# 株価データ取得
print("株価データを取得中...")
prices = fetch_stock_prices(start="2010-01-01")
print(f"取得銘柄数: {len(prices.columns)}")
print(f"期間: {prices.index[0].strftime('%Y-%m')} 〜 {prices.index[-1].strftime('%Y-%m')}")
prices.tail()

In [ ]:
# ========================================
# Chart 2: 全銘柄の正規化株価推移
# ========================================

prices_norm = normalize_prices(prices)

fig2 = go.Figure()

colors = px.colors.qualitative.Set2
for i, col in enumerate(prices_norm.columns):
    fig2.add_trace(
        go.Scatter(
            x=prices_norm.index,
            y=prices_norm[col],
            name=col,
            line=dict(color=colors[i % len(colors)], width=2),
            hovertemplate=f"{col}<br>" + "%{x|%Y-%m}<br>%{y:.0f}<extra></extra>",
        )
    )

fig2.update_layout(
    height=600, width=1100,
    template="plotly_white",
    title_text="日本半導体関連銘柄 株価推移（2010年初=100）",
    title_x=0.5,
    yaxis_title="正規化株価 (基準=100)",
    yaxis_type="log",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig2.show()

## 3. 半導体サイクルと株価の重ね合わせ

SI比率（サイクル指標）と個別銘柄の株価を2軸チャートで重ねる。
サイクルの転換点と株価のピーク/ボトムの時間差（リード/ラグ）を確認。

In [ ]:
# ========================================
# Chart 3: SI比率 vs 個別銘柄（2軸）
# ========================================

# 代表銘柄を選択
target_stocks = ["東京エレクトロン", "アドバンテスト", "レーザーテック", "ディスコ"]
available = [s for s in target_stocks if s in prices.columns]

fig3 = make_subplots(
    rows=len(available), cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=[f"{name} vs 半導体サイクル" for name in available],
    specs=[[{"secondary_y": True}]] * len(available),
)

for i, name in enumerate(available, 1):
    # 株価（左軸）
    fig3.add_trace(
        go.Scatter(
            x=prices.index, y=prices[name],
            name=f"{name} 株価",
            line=dict(color="#2196F3", width=2),
            showlegend=(i == 1),
        ),
        row=i, col=1, secondary_y=False,
    )
    # SI比率（右軸）
    fig3.add_trace(
        go.Scatter(
            x=iip.index, y=iip["si_ratio_ma3"],
            name="SI比率",
            line=dict(color="#FF5722", width=1.5, dash="dot"),
            opacity=0.8,
            showlegend=(i == 1),
        ),
        row=i, col=1, secondary_y=True,
    )
    fig3.add_hline(y=1.0, line_dash="dash", line_color="gray",
                   row=i, col=1, secondary_y=True)
    fig3.update_yaxes(title_text="株価(円)", row=i, col=1, secondary_y=False)
    fig3.update_yaxes(title_text="SI比率", row=i, col=1, secondary_y=True)

fig3.update_layout(
    height=300 * len(available),
    width=1100,
    template="plotly_white",
    title_text="半導体サイクル（SI比率） vs 主要銘柄株価",
    title_x=0.5,
    hovermode="x unified",
)

fig3.show()

## 4. サイクル局面別リターン分析

各サイクル局面（回復期・好況期・後退期・不況期）における
銘柄ごとの平均月次リターンを算出。

In [ ]:
# ========================================
# Chart 4: サイクル局面別の平均月次リターン
# ========================================

# 月次リターン算出
returns = prices.pct_change() * 100  # パーセント表示

# IIPのサイクル局面とマージ
returns_with_phase = returns.copy()
returns_with_phase["cycle_phase"] = iip["cycle_phase"].reindex(returns.index, method="ffill")
returns_with_phase = returns_with_phase.dropna(subset=["cycle_phase"])

# 局面別平均リターン
phase_order = ["回復期", "好況期", "後退期", "不況期"]
phase_returns = (
    returns_with_phase
    .groupby("cycle_phase")
    [available]
    .mean()
    .reindex(phase_order)
)

print("サイクル局面別 平均月次リターン (%)")
print(phase_returns.round(2))

# ヒートマップ
fig4 = go.Figure(
    go.Heatmap(
        z=phase_returns.values,
        x=phase_returns.columns,
        y=phase_returns.index,
        colorscale="RdYlGn",
        zmid=0,
        text=phase_returns.round(2).values,
        texttemplate="%{text}%",
        textfont={"size": 14},
        hovertemplate="%{y}<br>%{x}: %{z:.2f}%<extra></extra>",
    )
)

fig4.update_layout(
    height=400, width=900,
    template="plotly_white",
    title_text="サイクル局面別 平均月次リターン (%)",
    title_x=0.5,
    xaxis_title="銘柄",
    yaxis_title="サイクル局面",
)

fig4.show()

## 5. 相関分析

SI比率の変化と各銘柄のリターンの相関（同月・1ヶ月先行・3ヶ月先行）を確認。
株価がサイクルに対してどれだけ先行するかを定量化。

In [ ]:
# ========================================
# Chart 5: リード/ラグ相関分析
# ========================================

si_change = iip["si_ratio_ma3"].pct_change()

# 各ラグでの相関を計算
lags = range(-6, 7)  # -6ヶ月〜+6ヶ月
corr_results = {}

for name in available:
    stock_ret = returns[name].dropna()
    corrs = []
    for lag in lags:
        shifted_si = si_change.shift(lag)
        merged = pd.concat([stock_ret, shifted_si], axis=1).dropna()
        if len(merged) > 12:
            corrs.append(merged.iloc[:, 0].corr(merged.iloc[:, 1]))
        else:
            corrs.append(np.nan)
    corr_results[name] = corrs

corr_df = pd.DataFrame(corr_results, index=list(lags))
corr_df.index.name = "lag_months"

fig5 = go.Figure()
for i, name in enumerate(available):
    fig5.add_trace(
        go.Scatter(
            x=list(lags),
            y=corr_df[name],
            name=name,
            mode="lines+markers",
            line=dict(color=colors[i % len(colors)], width=2),
        )
    )

fig5.add_hline(y=0, line_dash="dash", line_color="gray")
fig5.add_vline(x=0, line_dash="dash", line_color="gray")

fig5.update_layout(
    height=500, width=900,
    template="plotly_white",
    title_text="株価リターンとSI比率変化の相関（リード/ラグ分析）",
    title_x=0.5,
    xaxis_title="ラグ（月）　←株価先行　　サイクル先行→",
    yaxis_title="相関係数",
    hovermode="x unified",
)

fig5.show()

# 最大相関のラグを表示
print("\n各銘柄の最大相関ラグ:")
for name in available:
    best_lag = corr_df[name].abs().idxmax()
    best_corr = corr_df[name].loc[best_lag]
    direction = "株価が先行" if best_lag < 0 else "サイクルが先行" if best_lag > 0 else "同時"
    print(f"  {name}: ラグ={best_lag}ヶ月 (相関={best_corr:.3f}, {direction})")

## 6. 現在のサイクル位置と投資示唆

直近のサイクル指標を確認し、過去の類似局面でのリターンから示唆を得る。

In [ ]:
# ========================================
# 現在のサイクル位置サマリー
# ========================================

latest = iip.iloc[-1]

print("=" * 50)
print("半導体サイクル 現在地サマリー")
print("=" * 50)
print(f"日付: {iip.index[-1].strftime('%Y年%m月')}")
print(f"出荷指数: {latest['shipment_index']:.1f}")
print(f"在庫指数: {latest['inventory_index']:.1f}")
print(f"SI比率: {latest['si_ratio']:.3f}")
print(f"SI比率(3MA): {latest['si_ratio_ma3']:.3f}")
print(f"出荷YoY: {latest['shipment_yoy']:+.1f}%")
print(f"在庫YoY: {latest['inventory_yoy']:+.1f}%")
print(f"局面判定: 【{latest['cycle_phase']}】")
print("=" * 50)

# 同一局面での過去リターン分布
current_phase = latest["cycle_phase"]
if current_phase != "不明":
    same_phase = returns_with_phase[returns_with_phase["cycle_phase"] == current_phase]
    print(f"\n{current_phase}における過去の月次リターン統計:")
    print(same_phase[available].describe().round(2))

In [ ]:
# ========================================
# Chart 6: サイクル時計（現在位置の可視化）
# ========================================

import plotly.graph_objects as go
import numpy as np

# サイクル局面を角度にマッピング
phase_angles = {
    "回復期": 45,    # 右上
    "好況期": 135,   # 左上
    "後退期": 225,   # 左下
    "不況期": 315,   # 右下
}

fig6 = go.Figure()

# 背景の4象限
for phase, angle in phase_angles.items():
    theta = np.radians(angle)
    fig6.add_annotation(
        x=1.1 * np.cos(theta), y=1.1 * np.sin(theta),
        text=f"<b>{phase}</b>",
        showarrow=False,
        font=dict(size=16),
    )

# 過去12ヶ月のSI比率推移を極座標的にプロット
recent = iip.tail(24)
si_vals = recent["si_ratio_ma3"].values
si_diff = np.diff(si_vals)

# x = SI比率水準（1.0が中心）、y = SI比率の変化方向
x_vals = si_vals[1:] - 1.0  # 1.0からの乖離
y_vals = si_diff  # 変化量

fig6.add_trace(
    go.Scatter(
        x=x_vals, y=y_vals,
        mode="lines+markers",
        marker=dict(
            size=np.linspace(4, 14, len(x_vals)),
            color=np.arange(len(x_vals)),
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="月", len=0.5),
        ),
        line=dict(color="rgba(100,100,100,0.3)"),
        text=[d.strftime("%Y-%m") for d in recent.index[1:]],
        hovertemplate="%{text}<br>SI比率-1: %{x:.3f}<br>SI変化: %{y:.3f}<extra></extra>",
        showlegend=False,
    )
)

# 現在位置を強調
fig6.add_trace(
    go.Scatter(
        x=[x_vals[-1]], y=[y_vals[-1]],
        mode="markers+text",
        marker=dict(size=20, color="red", symbol="star"),
        text=["現在"],
        textposition="top center",
        textfont=dict(size=14, color="red"),
        showlegend=False,
    )
)

# 象限の区切り線
fig6.add_hline(y=0, line_dash="dash", line_color="gray")
fig6.add_vline(x=0, line_dash="dash", line_color="gray")

# 象限ラベル
fig6.add_annotation(x=0.15, y=0.015, text="好況期", showarrow=False,
                    font=dict(size=20, color="rgba(33,150,243,0.3)"))
fig6.add_annotation(x=-0.15, y=0.015, text="回復期", showarrow=False,
                    font=dict(size=20, color="rgba(76,175,80,0.3)"))
fig6.add_annotation(x=0.15, y=-0.015, text="後退期", showarrow=False,
                    font=dict(size=20, color="rgba(255,152,0,0.3)"))
fig6.add_annotation(x=-0.15, y=-0.015, text="不況期", showarrow=False,
                    font=dict(size=20, color="rgba(244,67,54,0.3)"))

fig6.update_layout(
    height=600, width=700,
    template="plotly_white",
    title_text="半導体サイクル時計（過去24ヶ月の軌跡）",
    title_x=0.5,
    xaxis_title="SI比率 − 1.0（水準）",
    yaxis_title="SI比率 変化量（方向）",
    xaxis=dict(zeroline=True),
    yaxis=dict(zeroline=True),
)

fig6.show()